In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

from braindecode.models import EEGNetv4
import copy
import random
import pandas as pd
from tqdm import notebook
import tqdm
from pathlib import Path
import numpy as np
import scipy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

# DNN definition

In [ ]:
class SSVEPDNN(nn.Module):
    def __init__(self, num_classes=40, channels=9, samples=250, subbands=3):
        super(SSVEPDNN, self).__init__()
        # [batch, subbands, channels, time]
        # Subband combination layer
        self.subband_combination = nn.Conv2d(
            subbands, 1, kernel_size=(1, 1), bias=False
        )
        # Channel combination layer
        self.channel_combination = nn.Conv2d(1, 120, kernel_size=(channels, 1))
        # First dropout
        self.drop1 = nn.Dropout(0.1)
        # Third layer - Time convolution
        self.third_conv = nn.Conv2d(120, 120, kernel_size=(1, 2), stride=(1, 2))
        # Second droput
        self.drop2 = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        # 4th conv - FIR filtering
        self.fourth_conv = nn.Conv2d(120, 120, kernel_size=(1, 10), padding="same")
        self.drop3 = nn.Dropout(0.95)

        # Fully connected layer - Classifier
        self.fc = nn.Linear(120 * (samples // 2), num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        with torch.no_grad():
            self.subband_combination.weight.fill_(1.0)

    def forward(self, x):
        # x shape: [batch, subbands, channels, time]
        x = self.subband_combination(x)  # [batch, 1, channels, time]
        x = self.channel_combination(x)  # [batch, 120, 1, time]
        x = self.drop1(x)
        x = self.third_conv(x)  # [batch, 120, 1, time/2]
        x = self.drop2(x)
        x = self.relu(x)
        x = self.fourth_conv(x)  # [batch, 120, 1, time/2]
        x = self.drop3(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # [batch, num_classes]
        output = F.softmax(x, dim=1)
        return output

# Utilities

In [ ]:
from cross_subject_utils import (
    plot_learning_curves,
    evaluate,
    get_windows,
    load_data_from_users,
    get_windows,
    filter_signals_subbands,
)

In [ ]:
from cca import CCA_otimizacao, matriz_referencia

In [ ]:
def train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=100,
    device=0,
    save_path="best_model.pth",
):
    best_val_accuracy = -float("inf")
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in tqdm.notebook.tqdm(range(num_epochs)):
        # Training Phase
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # eval train
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_accuracy = train_correct / train_total
        avg_train_loss = running_loss / len(train_loader)

        # eval validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # val accuracy
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        # Save if best vall acc
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), save_path)
            # print(f"Best model saved with accuracy: {best_val_accuracy:.4f}")

        print(
            f"Epoch {epoch + 1}/{num_epochs}: "
            f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
            f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
        )
    plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies)
    model.load_state_dict(best_model)
    return model

# Cross Subject

In [ ]:
freq_phase_path = (
    "C:/Users/machi/Documents/Mestrado/repos/data/benchmark/Freq_Phase.mat"
)
freq_phase = scipy.io.loadmat(freq_phase_path)
frequencias = np.round(freq_phase["freqs"], 2).ravel()
fases = freq_phase["phases"]

# Parâmetros do pré-processamento
sample_rate = 250
delay = 160

# Parâmetros do CCA
num_harmonica = 5
inform_fase = 0

# Parâmetros de janelas e sessões
tamanho_da_janela_seg = 1
tamanho_da_janela = int(np.ceil(tamanho_da_janela_seg * sample_rate))

occipital_electrodes = np.array([47, 53, 54, 55, 56, 57, 60, 61, 62])
frequencias_desejadas = frequencias[:8]
indices = [np.where(frequencias == freq)[0][0] for freq in frequencias_desejadas]

# Usuários
# users = list(range(1, 36))  # Usuários de 1 a 35
users = list(range(1, 11))  # Usuários de 1 a 10

epochs = 300
exp_dir = Path(
    f"CCA_dnn_3_subband_independent/{len(users)}_users_{len(frequencias_desejadas)}_freqs_{tamanho_da_janela_seg}_s/"
)

In [ ]:
print("Usuários de interesse:", users)
print(f"Frequencies used: {frequencias_desejadas}")
print(f"Frequencies indices: {indices}")

In [ ]:
all_data = load_data_from_users(
    users,
    visual_delay=delay,
    filter_bandpass=False,
    sample_rate=sample_rate,
)

# Training

In [ ]:
seed = 42
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

# Cross-Subject EEGNet Training (single window per trial, no window separation)
metricas_usuarios = []
exp_dir.mkdir(parents=True, exist_ok=True)

# Prepare cross-subject splits
for test_user_idx, test_user in enumerate(users):
    print(f"Processando Usuário {test_user}")
    train_users = [u for u in users if u != test_user]
    print(f"Train Users: {train_users}")

    # Concatenate training data from all train_users
    train_data = np.concatenate(
        [all_data[users.index(u)] for u in train_users], axis=-1
    )  # shape: (channels, samples, freqs, trials)
    test_data = all_data[test_user_idx]

    num_canais, _, num_freqs, num_trials_train = train_data.shape
    num_trials_test = test_data.shape[-1]

    # Prepare reference matrices for all frequencies (no window separation)
    Y_train = np.zeros(
        (tamanho_da_janela * num_trials_train, num_harmonica * 2, len(indices))
    )
    Y_test = np.zeros(
        (tamanho_da_janela * num_trials_test, num_harmonica * 2, len(indices))
    )
    for k in indices:
        print(k)
        print(
            f"Generating reference for frequency index {k}, frequency {frequencias[k]} Hz"
        )
        y_train = matriz_referencia(
            num_harmonica,
            inform_fase,
            num_trials_train,
            frequencias[k],
            fases,
            tamanho_da_janela,
        )
        Y_train[:, :, k] = y_train
        print(f"Y_train shape after freq {frequencias[k]} Hz: {Y_train.shape}")
        y_test = matriz_referencia(
            num_harmonica,
            inform_fase,
            num_trials_test,
            frequencias[k],
            fases,
            tamanho_da_janela,
        )
        Y_test[:, :, k] = y_test
        print(f"Y_test shape after freq {frequencias[k]} Hz: {Y_test.shape}")

    X_train = np.zeros(
        (
            tamanho_da_janela * num_trials_train,
            3,
            len(occipital_electrodes),
            len(indices),
        )
    )
    X_test = np.zeros(
        (
            tamanho_da_janela * num_trials_test,
            3,
            len(occipital_electrodes),
            len(indices),
        )
    )

    for k in range(len(indices)):
        # For training: each trial is a single window
        eeg_matrix_train = train_data[
            occipital_electrodes, :tamanho_da_janela, indices[k], :
        ].reshape(-1, len(occipital_electrodes), tamanho_da_janela)
        eeg_matrix_train = filter_signals_subbands(
            eeg_matrix_train, subban_no=3, sampling_rate=250
        )
        eeg_matrix_test = test_data[
            occipital_electrodes, :tamanho_da_janela, indices[k], :
        ].reshape(-1, len(occipital_electrodes), tamanho_da_janela)
        eeg_matrix_test = filter_signals_subbands(
            eeg_matrix_test, subban_no=3, sampling_rate=250
        )

        eeg_matrix_train = np.moveaxis(eeg_matrix_train, -1, 0)
        eeg_matrix_test = np.moveaxis(eeg_matrix_test, -1, 0)
        eeg_matrix_train = np.concatenate(eeg_matrix_train, axis=0)
        eeg_matrix_test = np.concatenate(eeg_matrix_test, axis=0)

        X_train[:, :, :, k] = eeg_matrix_train
        X_test[:, :, :, k] = eeg_matrix_test

    # # CCA optimization (across all training data)
    Combinadores_Y = []
    Combinadores_X = []
    for i in range(3):
        Combinadores_Y_sub = []
        Combinadores_X_sub = []
        for k in range(len(indices)):
            Wx, Wy, _ = CCA_otimizacao(X_train[:, i, :, k], Y_train[:, :, k])
            Combinadores_Y_sub.append(Wy)
            Combinadores_X_sub.append(Wx)
        Combinadores_X.append(np.column_stack(Combinadores_X_sub))
        Combinadores_Y.append(np.column_stack(Combinadores_Y_sub))
    Combinadores_X = np.array(Combinadores_X)  # shape: (3, len(indices), channels)
    Combinadores_Y = np.array(
        Combinadores_Y
    )  # shape: (3, len(indices), 2*num_harmonica)

    # Separar em janelas
    X_teste_janelas = []
    X_treino_janelas = []
    Y_teste_janelas = []
    Y_treino_janelas = []

    for k in range(len(indices)):
        X_t, numero_janelas_teste = get_windows(
            X_test[:, :, :, k], tamanho_da_janela, include_last=False
        )
        Y_t, _ = get_windows(Y_test[:, :, k], tamanho_da_janela, include_last=False)

        X_v, numero_janelas_treino = get_windows(
            X_train[:, :, :, k], tamanho_da_janela, include_last=False
        )
        Y_v, _ = get_windows(Y_train[:, :, k], tamanho_da_janela, include_last=False)

        X_teste_janelas.append(X_t)
        Y_teste_janelas.append(Y_t)

        X_treino_janelas.append(X_v)
        Y_treino_janelas.append(Y_v)

    # Construir tensor de treinamento
    rotulos_treinamento = []
    tensor_treinamento = np.zeros(
        [len(indices) * num_trials_train, 3, len(indices), tamanho_da_janela]
    )
    cont = 0

    for m in range(len(indices)):
        for j in range(numero_janelas_treino):
            rotulos_treinamento.append(frequencias[indices[m]])
            cont_1 = 0
            for w in range(len(indices)):
                for subband in range(3):
                    Wx = Combinadores_X[subband, :, w]
                    janela_x = X_treino_janelas[m][j][:, subband, :]
                    projecao_x = np.dot(Wx, janela_x.T)
                    tensor_treinamento[cont, subband, cont_1, :] = projecao_x

                cont_1 += 1
            cont += 1

    rotulos_teste = []
    tensor_teste = np.zeros(
        [len(indices) * num_trials_test, 3, len(indices), tamanho_da_janela]
    )
    cont = 0

    for m in range(len(indices)):
        for j in range(numero_janelas_teste):
            rotulos_teste.append(frequencias[indices[m]])
            cont_1 = 0
            for w in range(len(indices)):
                for subband in range(3):
                    Wx = Combinadores_X[subband, :, w]
                    janela_x = X_teste_janelas[m][j][:, subband, :]
                    projecao_x = np.dot(Wx, janela_x.T)
                    tensor_teste[cont, subband, cont_1, :] = projecao_x

                cont_1 += 1
            cont += 1

    # Map labels to indices
    mapeamento = {rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))}
    rotulos_treinamento = torch.tensor(
        [
            mapeamento[rotulo.item()] if hasattr(rotulo, "item") else mapeamento[rotulo]
            for rotulo in rotulos_treinamento
        ]
    )
    rotulos_teste = torch.tensor(
        [
            mapeamento[rotulo.item()] if hasattr(rotulo, "item") else mapeamento[rotulo]
            for rotulo in rotulos_teste
        ]
    )

    X_treino = torch.tensor(tensor_treinamento, dtype=torch.float32).to(device)
    X_teste = torch.tensor(tensor_teste, dtype=torch.float32).to(device)
    Y_treino = torch.tensor(rotulos_treinamento, dtype=torch.long).to(device)
    Y_teste = torch.tensor(rotulos_teste, dtype=torch.long).to(device)
    print("X_train:", X_treino.shape)
    print("X_test:", X_teste.shape)
    print("Y_train:", Y_treino.shape)
    print("Y_test:", Y_teste.shape)

    # Model setup
    model = SSVEPDNN(
        num_classes=len(frequencias_desejadas),
        channels=8,
        samples=tamanho_da_janela,
        subbands=3,
    )
    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    dataset = TensorDataset(X_treino, Y_treino)
    train_size = int(0.85 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(
        dataset, [train_size, val_size], generator=torch.Generator().manual_seed(seed)
    )
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(
        TensorDataset(X_teste, Y_teste), batch_size=10, shuffle=False
    )

    # Train
    best_model = train(
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        num_epochs=epochs,
        device=device,
        save_path=exp_dir.joinpath(f"best_model_user_{test_user}.pth"),
    )

    # Evaluate
    accuracy, recall, f1, cm = evaluate(best_model, test_loader)

    metricas_usuarios.append(
        {
            "usuario": test_user,
            "acuracia": accuracy,
            "recall": recall,
            "f1-score": f1,
            "confusion_matrix": cm,
        }
    )
    print(
        f"Test User {test_user} Finished: Accuracy={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
    )

    # Salvar as métricas de cada usuário
    df_metricas = pd.DataFrame(metricas_usuarios)
    df_metricas.to_csv(exp_dir.joinpath("metricas.csv"), index=False)

    print("-" * 50)

In [ ]:
# seed = 42
# torch.cuda.manual_seed(seed)
# torch.manual_seed(seed)

# # Cross-Subject EEGNet Training (single window per trial, no window separation)
# metricas_usuarios = []
# exp_dir.mkdir(parents=True, exist_ok=True)

# # Prepare cross-subject splits
# for test_user_idx, test_user in enumerate(users):
#     print(f"Processando Usuário {test_user}")
#     train_users = [u for u in users if u != test_user]
#     print(f"Train Users: {train_users}")

#     # Concatenate training data from all train_users
#     train_data = np.concatenate(
#         [all_data[users.index(u)] for u in train_users], axis=-1
#     )  # shape: (channels, samples, freqs, trials)
#     test_data = all_data[test_user_idx]

#     num_canais, _, num_freqs, num_trials_train = train_data.shape
#     num_trials_test = test_data.shape[-1]

#     # Prepare reference matrices for all frequencies (no window separation)
#     Y_train = np.zeros(
#         (tamanho_da_janela * num_trials_train, num_harmonica * 2, len(indices))
#     )
#     Y_test = np.zeros(
#         (tamanho_da_janela * num_trials_test, num_harmonica * 2, len(indices))
#     )
#     for k in indices:
#         print(k)
#         print(
#             f"Generating reference for frequency index {k}, frequency {frequencias[k]} Hz"
#         )
#         y_train = matriz_referencia(
#             num_harmonica,
#             inform_fase,
#             num_trials_train,
#             frequencias[k],
#             fases,
#             tamanho_da_janela,
#         )
#         Y_train[:, :, k] = y_train
#         print(f"Y_train shape after freq {frequencias[k]} Hz: {Y_train.shape}")
#         y_test = matriz_referencia(
#             num_harmonica,
#             inform_fase,
#             num_trials_test,
#             frequencias[k],
#             fases,
#             tamanho_da_janela,
#         )
#         Y_test[:, :, k] = y_test
#         print(f"Y_test shape after freq {frequencias[k]} Hz: {Y_test.shape}")

#     X_train = np.zeros(
#         (tamanho_da_janela * num_trials_train, len(occipital_electrodes), len(indices))
#     )
#     X_test = np.zeros(
#         (tamanho_da_janela * num_trials_test, len(occipital_electrodes), len(indices))
#     )

#     for k in range(len(indices)):
#         # For training: each trial is a single window
#         eeg_matrix_train = train_data[
#             occipital_electrodes, :tamanho_da_janela, indices[k], :
#         ]
#         # filter_signals_subbands(eeg_matrix_train, subban_no=3, sampling_rate=250)
#         eeg_matrix_test = test_data[
#             occipital_electrodes, :tamanho_da_janela, indices[k], :
#         ]
#         # filter_signals_subbands(eeg_matrix_test, subban_no=3, sampling_rate=250)

#         # Transpõe os dados para que cada linha represente uma amostra
#         eeg_matrix_train = np.transpose(eeg_matrix_train)
#         eeg_matrix_test = np.transpose(eeg_matrix_test)
#         eeg_matrix_train = np.concatenate(eeg_matrix_train, axis=0)
#         eeg_matrix_test = np.concatenate(eeg_matrix_test, axis=0)

#         X_train[:, :, k] = eeg_matrix_train
#         X_test[:, :, k] = eeg_matrix_test

#     # CCA optimization (across all training data)
#     Combinadores_Y = []
#     Combinadores_X = []
#     correlacoes_max = []
#     for k in range(len(indices)):
#         Wx, Wy, corr = CCA_otimizacao(X_train[:, :, k], Y_train[:, :, k])
#         Combinadores_Y.append(Wy)
#         Combinadores_X.append(Wx)
#         correlacoes_max.append(corr)
#     Combinadores_X = np.column_stack(Combinadores_X)
#     Combinadores_Y = np.column_stack(Combinadores_Y)

#     # Separar em janelas
#     X_teste_janelas = []
#     X_treino_janelas = []
#     Y_teste_janelas = []
#     Y_treino_janelas = []

#     for k in range(len(indices)):
#         X_t, numero_janelas_teste = get_windows(
#             X_test[:, :, k], tamanho_da_janela, include_last=False
#         )
#         Y_t, _ = get_windows(Y_test[:, :, k], tamanho_da_janela, include_last=False)

#         X_v, numero_janelas_treino = get_windows(
#             X_train[:, :, k], tamanho_da_janela, include_last=False
#         )
#         Y_v, _ = get_windows(Y_train[:, :, k], tamanho_da_janela, include_last=False)

#         X_teste_janelas.append(X_t)
#         Y_teste_janelas.append(Y_t)

#         X_treino_janelas.append(X_v)
#         Y_treino_janelas.append(Y_v)

#     # Construir tensor de treinamento
#     rotulos_treinamento = []
#     tensor_treinamento = np.zeros(
#         [len(indices) * num_trials_train, len(indices), tamanho_da_janela]
#     )
#     cont = 0

#     for m in range(len(indices)):
#         for j in range(numero_janelas_treino):
#             janela_x = X_treino_janelas[m][j]
#             rotulos_treinamento.append(frequencias[indices[m]])
#             cont_1 = 0
#             for w in range(len(indices)):
#                 Wx = Combinadores_X[:, w]

#                 projecao_x = np.dot(Wx, janela_x.T)
#                 tensor_treinamento[cont, cont_1, :] = projecao_x

#                 cont_1 += 1
#             cont += 1

#     rotulos_teste = []
#     tensor_teste = np.zeros(
#         [len(indices) * num_trials_test, len(indices), tamanho_da_janela]
#     )
#     cont = 0

#     for m in range(len(indices)):
#         for j in range(numero_janelas_teste):
#             janela_x = X_teste_janelas[m][j]
#             rotulos_teste.append(frequencias[indices[m]])
#             cont_1 = 0

#             for w in range(len(indices)):
#                 Wx = Combinadores_X[:, w]

#                 projecao_x = np.dot(Wx, janela_x.T)
#                 tensor_teste[cont, cont_1, :] = projecao_x

#                 cont_1 += 1
#             cont += 1

#     # Map labels to indices
#     mapeamento = {rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))}
#     rotulos_treinamento = torch.tensor(
#         [
#             mapeamento[rotulo.item()] if hasattr(rotulo, "item") else mapeamento[rotulo]
#             for rotulo in rotulos_treinamento
#         ]
#     )
#     rotulos_teste = torch.tensor(
#         [
#             mapeamento[rotulo.item()] if hasattr(rotulo, "item") else mapeamento[rotulo]
#             for rotulo in rotulos_teste
#         ]
#     )
#     tensor_treinamento = filter_signals_subbands(
#         tensor_treinamento, subban_no=1, sampling_rate=250
#     )
#     tensor_teste = filter_signals_subbands(tensor_teste, subban_no=1, sampling_rate=250)

#     X_treino = torch.tensor(tensor_treinamento, dtype=torch.float32).to(device)
#     X_teste = torch.tensor(tensor_teste, dtype=torch.float32).to(device)
#     Y_treino = torch.tensor(rotulos_treinamento, dtype=torch.long).to(device)
#     Y_teste = torch.tensor(rotulos_teste, dtype=torch.long).to(device)
#     print("X_train:", X_treino.shape)
#     print("X_test:", X_teste.shape)
#     print("Y_train:", Y_treino.shape)
#     print("Y_test:", Y_teste.shape)

#     # Model setup
#     model = SSVEPDNN(
#         num_classes=len(frequencias_desejadas),
#         channels=8,
#         samples=tamanho_da_janela,
#         subbands=1,
#     )
#     model = model.to(device)
#     criterion = torch.nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#     dataset = TensorDataset(X_treino, Y_treino)
#     train_size = int(0.85 * len(dataset))
#     val_size = len(dataset) - train_size
#     train_dataset, val_dataset = random_split(
#         dataset, [train_size, val_size], generator=torch.Generator().manual_seed(seed)
#     )
#     train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
#     test_loader = DataLoader(
#         TensorDataset(X_teste, Y_teste), batch_size=10, shuffle=False
#     )

#     # Train
#     best_model = train(
#         model,
#         train_loader,
#         val_loader,
#         criterion,
#         optimizer,
#         num_epochs=epochs,
#         device=device,
#         save_path=exp_dir.joinpath(f"best_model_user_{test_user}.pth"),
#     )

#     # Evaluate
#     accuracy, recall, f1, cm = evaluate(best_model, test_loader)

#     metricas_usuarios.append(
#         {
#             "usuario": test_user,
#             "acuracia": accuracy,
#             "recall": recall,
#             "f1-score": f1,
#             "confusion_matrix": cm,
#         }
#     )
#     print(
#         f"Test User {test_user} Finished: Accuracy={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
#     )

#     # Salvar as métricas de cada usuário
# df_metricas = pd.DataFrame(metricas_usuarios)
#     df_metricas.to_csv(exp_dir.joinpath("metricas.csv"), index=False)

#     print("-" * 50)